## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts()>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1800
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < cutoff].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [20]:
# Optimization 1 - doing feature engineering on ASK_AMT column
application_df['ASK_AMT'].value_counts()

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [21]:
# Finding the max value to see the upper limit for binning
application_df['ASK_AMT'].max()

8597806340

In [22]:
# Finding the min value to see the lower limit for binning
application_df['ASK_AMT'].min()

5000

In [23]:
# Define custom bin edges
bins=[0, 5000, 20000, 50000, 100000, 500000, 1000000, 10000000, 50000000, 100000000, 500000000, 1000000000, float('inf')]
# Define custom bin labels
bin_labels = ['upto 5k', '5-20k','20-50k','50-100k','100-500k','500k-1M','1-10M','10-50M','50-100M','100-500M','500M-1B','> 1B']
# Create a different dataframe so that our application_df doesn't get spoiled.
ask_amt_df=application_df.copy()
# Perform custom binning
ask_amt_df['ASK_AMT_Binned'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=bin_labels, include_lowest=True)
ask_amt_df['ASK_AMT_Binned'].value_counts()

ASK_AMT_Binned
upto 5k     25398
100-500k     2304
20-50k       1609
50-100k      1423
5-20k        1338
1-10M        1165
500k-1M       650
10-50M        258
100-500M       62
50-100M        53
500M-1B        21
> 1B           18
Name: count, dtype: int64

In [15]:
ask_amt_df = ask_amt_df.drop(['ASK_AMT'], axis=1)
ask_amt_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_Binned
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1,upto 5k
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,100-500k
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,upto 5k
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,5-20k
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,100-500k


In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
success_ask_df = pd.get_dummies(ask_amt_df).astype(int)
success_ask_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ASK_AMT_Binned_20-50k,ASK_AMT_Binned_50-100k,ASK_AMT_Binned_100-500k,ASK_AMT_Binned_500k-1M,ASK_AMT_Binned_1-10M,ASK_AMT_Binned_10-50M,ASK_AMT_Binned_50-100M,ASK_AMT_Binned_100-500M,ASK_AMT_Binned_500M-1B,ASK_AMT_Binned_> 1B
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [25]:
# Split our preprocessed data into our features and target arrays
y = success_ask_df['IS_SUCCESSFUL'].values
X = success_ask_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                1792      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2337 (9.13 KB)
Trainable params: 2337 (9.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,batch_size = 250)

Epoch 1/50
103/103 [==============================] - 1s 3ms/step - loss: 0.6312 - accuracy: 0.6784
Epoch 2/50
103/103 [==============================] - 1s 5ms/step - loss: 0.5711 - accuracy: 0.7229
Epoch 3/50
103/103 [==============================] - 0s 5ms/step - loss: 0.5608 - accuracy: 0.7268
Epoch 4/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5562 - accuracy: 0.7297
Epoch 5/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5536 - accuracy: 0.7310
Epoch 6/50
103/103 [==============================] - 0s 5ms/step - loss: 0.5507 - accuracy: 0.7325
Epoch 7/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5495 - accuracy: 0.7344
Epoch 8/50
103/103 [==============================] - 0s 3ms/step - loss: 0.5482 - accuracy: 0.7343
Epoch 9/50
103/103 [==============================] - 0s 3ms/step - loss: 0.5468 - accuracy: 0.7352
Epoch 10/50
103/103 [==============================] - 0s 3ms/step - loss: 0.5459 - accuracy: 0.7361

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5589 - accuracy: 0.7222 - 537ms/epoch - 2ms/step
Loss: 0.5588799715042114, Accuracy: 0.7222157716751099


In [34]:
# ASK_AMT feature engineering reduced the accuracy and increased the loss. Hence sticking with the original dataframe with ASK_AMT column.
# Convert categorical data to numeric with `pd.get_dummies`
success_df = pd.get_dummies(application_df).astype(int)
success_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [35]:
# Split our preprocessed data into our features and target arrays
y = success_df['IS_SUCCESSFUL'].values
X = success_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Optimization 2 - Adding another hidden layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Thirs hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                2816      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5441 (21.25 KB)
Trainable params: 5441 (21.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,batch_size = 250)

Epoch 1/50
103/103 [==============================] - 2s 5ms/step - loss: 0.5898 - accuracy: 0.7067
Epoch 2/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5543 - accuracy: 0.7292
Epoch 3/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5508 - accuracy: 0.7315
Epoch 4/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5484 - accuracy: 0.7334
Epoch 5/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5477 - accuracy: 0.7349
Epoch 6/50
103/103 [==============================] - 0s 4ms/step - loss: 0.5473 - accuracy: 0.7339
Epoch 7/50
103/103 [==============================] - 1s 5ms/step - loss: 0.5455 - accuracy: 0.7350
Epoch 8/50
103/103 [==============================] - 1s 5ms/step - loss: 0.5448 - accuracy: 0.7351
Epoch 9/50
103/103 [==============================] - 1s 5ms/step - loss: 0.5440 - accuracy: 0.7356
Epoch 10/50
103/103 [==============================] - 1s 5ms/step - loss: 0.5443 - accuracy: 0.7356

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5552 - accuracy: 0.7237 - 518ms/epoch - 2ms/step
Loss: 0.5551714897155762, Accuracy: 0.7237317562103271


In [41]:
#Adding one more hidden layer decreased the accuracy. So I am going to stay with 2 layers.
#Optimization 3 - Change the activation function
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="swish"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="swish"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 32)                1408      
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1953 (7.63 KB)
Trainable params: 1953 (7.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [43]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50, batch_size = 250)

Epoch 1/50
103/103 [==============================] - 1s 2ms/step - loss: 0.6404 - accuracy: 0.6607
Epoch 2/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5765 - accuracy: 0.7224
Epoch 3/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5647 - accuracy: 0.7275
Epoch 4/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5593 - accuracy: 0.7308
Epoch 5/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5563 - accuracy: 0.7309
Epoch 6/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5536 - accuracy: 0.7321
Epoch 7/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5521 - accuracy: 0.7328
Epoch 8/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5513 - accuracy: 0.7327
Epoch 9/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5504 - accuracy: 0.7331
Epoch 10/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5493 - accuracy: 0.7338

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7247 - 503ms/epoch - 2ms/step
Loss: 0.5537082552909851, Accuracy: 0.7246647477149963


In [45]:
# Accuracy increased a tiny bit but still not close to our desired accuracy. Optimization 4 - Increase the number of neurons
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 32

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="swish")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="swish"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 64)                2816      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4929 (19.25 KB)
Trainable params: 4929 (19.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model - reduce the epochs size
fit_model = nn.fit(X_train_scaled,y_train,epochs=50, batch_size = 250)

Epoch 1/50
103/103 [==============================] - 1s 2ms/step - loss: 0.5988 - accuracy: 0.7055
Epoch 2/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5628 - accuracy: 0.7261
Epoch 3/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5574 - accuracy: 0.7289
Epoch 4/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5551 - accuracy: 0.7292
Epoch 5/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5533 - accuracy: 0.7310
Epoch 6/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5520 - accuracy: 0.7319
Epoch 7/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5500 - accuracy: 0.7331
Epoch 8/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5494 - accuracy: 0.7327
Epoch 9/50
103/103 [==============================] - 0s 2ms/step - loss: 0.5486 - accuracy: 0.7329
Epoch 10/50
103/103 [==============================] - 0s 3ms/step - loss: 0.5476 - accuracy: 0.7336

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5538 - accuracy: 0.7269 - 507ms/epoch - 2ms/step
Loss: 0.5538239479064941, Accuracy: 0.7268804907798767


In [ ]:
# Was able to increase the accuracy upto 72.68%. Could not reach the desired accuracy of 75%.

In [48]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
from google.colab import files
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>